In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = load_breast_cancer()

In [3]:
df = pd.DataFrame(data.data,columns=data.feature_names)

In [4]:
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [5]:
X,y = load_breast_cancer(return_X_y=True)

In [6]:
 X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.tree import DecisionTreeClassifier

In [8]:
model = DecisionTreeClassifier(max_depth=5)

In [9]:
model.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5)

In [10]:
model.score(X_test,y_test)

0.9468085106382979

In [11]:
class Node:
    def __init__(self,feature=None,threshold=None,left=None,right=None,value=None):
        self.feature = feature
        self.threshold=threshold
        self.value = value
        self.left = left
        self.right = right
    def isleaf(self):
        return self.value != None
    
class DecisionTree():
    def __init__(self,min_samples_split=2,max_depth=100,nfeatures=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.nfeatures = nfeatures
        self.root = None
        
    def fit(self,X,y):
        self.nfeatures = X.shape[1] if not self.nfeatures else min(X.shape[1],self.nfeatures)
        self.root = self._build_tree(X,y)
    
    def _build_tree(self,X,y,depth=0):
        n_samples,n_features = X.shape
        n_labels = np.unique(y)
        
        if depth >= self.max_depth or len(n_labels) <= 1 or n_samples < self.min_samples_split:
            leaf_value = self._value(y)
            return Node(value=leaf_value)
        
        feature_idxs = np.random.choice(n_features,self.nfeatures,replace=False)
        threshold,feature = self._bestsplit(X,y,feature_idxs)
        
        left_idxs,right_idxs = self._split(X[:,feature],threshold)
        
        left = self._build_tree(X[left_idxs,:],y[left_idxs],depth+1)
        right = self._build_tree(X[right_idxs,:],y[right_idxs],depth+1)
        
        return Node(feature,threshold,left,right)
    
    def _bestsplit(self,X,y,feature_idxs):
        best_gain = -1
        split_idx,split_threshold = None,None
        
        for idx in feature_idxs:
            X_col = X[:,idx]
            thresholds = np.unique(X_col)
            
            for threshold in thresholds:
                gain = self._infogain(X_col,y,threshold)
                
                if gain > best_gain:
                    best_gain = gain
                    split_idx,split_threshold = idx,threshold
                    
        return split_threshold,split_idx
    
    def _infogain(self,X_col,y,threshold):
        
        parent_entropy = self._entropy(y)
        
        left_idx,right_idx = self._split(X_col,threshold)
        
        if len(left_idx) == 0 or len(right_idx) == 0:
            return 0
        
        n = len(y)
        
        left_n = len(left_idx)
        right_n = len(right_idx)
        
        ent_l,ent_r = self._entropy(y[left_idx]),self._entropy(y[right_idx])
        
        child_entropy = (left_n/n) * ent_l + (right_n/n) * ent_r
        
        
        information_gain = parent_entropy - child_entropy
        
        return information_gain
        
        
    def _entropy(self,y):
        labels = np.bincount(y)
        probs = labels/len(y)
        entropies = [p*np.log(p) for p in probs if p > 0]
        return -np.sum(entropies)
        
    def _split(self,X_col,threshold):
        left = np.argwhere(X_col<=threshold).flatten()
        right = np.argwhere(X_col>threshold).flatten()
        return left,right
    
        
    def _value(self,y):
        labels,counts = np.unique(y,return_counts=True)
        return labels[np.argmax(counts)]
    
    def predict(self,X):
        return np.array([self._traverse(x,self.root) for x in X])
    
    def _traverse(self,x,node):
        if node.isleaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse(x,node.left)
        else:
            return self._traverse(x,node.right)
    def score(self, X, y):
            yp = self.predict(X)
            return np.mean(yp == y)

In [12]:
model = DecisionTree(max_depth=5)

In [13]:
model.fit(X_train,y_train)

In [14]:
model.score(X_test,y_test)

0.9521276595744681